# Common Functions

In [66]:
import twitter

def oauth_login():
    # XXX: Go to  to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://developer.twitter.com/en/docs/basics/authentication/overview/oauth
    # for more information on Twitter's OAuth implementation.
    
    CONSUMER_KEY = ''
    CONSUMER_SECRET = ''
    OAUTH_TOKEN = ''
    OAUTH_TOKEN_SECRET = ''
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

In [58]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [57]:
from textblob import TextBlob
def get_tweet_sentiment(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [46]:
def twitter_search(twitter_api, q, max_results=200, **kw):

    # See https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets
    # and https://developer.twitter.com/en/docs/tweets/search/guides/standard-operators
    # for details on advanced search criteria that may be useful for 
    # keyword arguments
    
    
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
    
    statuses = search_results['statuses']
    # Iterate through batches of results by following the cursor until we
    # reach the desired number of results, keeping in mind that OAuth users
    # can "only" make 180 search queries per 15-minute interval. See
    # https://developer.twitter.com/en/docs/basics/rate-limits
    # for details. A reasonable number of results is ~1000, although
    # that number of results may not exist for all queries.
    
    # Enforce a reasonable limit
    # Read this https://developer.twitter.com/en/docs/tweets/timelines/guides/working-with-timelines to understand why we use 
    # max_id to populate tweets
    max_results = min(1000, max_results)
    
    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break
            
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        
        if len(statuses) > max_results: 
            break
            
    return statuses

In [63]:
def remove_retweets(results):
    tweets = []
    for i in results:
        if i['text'] not in tweets:
            tweets.append(i['text'])
    return tweets

# Basic Sentiment Analysis

In [65]:
twitter_api = oauth_login()
stock_names = ['$TERA', '$ARMC', '$GTMM', '$GNCP', '$GLAE', '$CRAY', '$GPRO', '$SWCH', '$SCKT']
for stock in stock_names:
    q = stock
    results = twitter_search(twitter_api, q, max_results=100)
    positive = 0
    negative = 0
    neutral = 0
    tweets = remove_retweets(results)
    for tweet in tweets:
        sentiment = get_tweet_sentiment(tweet)
        if sentiment == 'positive':
            positive += 1
        elif sentiment == 'negative':
            negative += 1
        else:
            neutral += 1
    print('Positive tweets for {} is {} %'.format(stock, (positive/len(results))*100))
    print('Neutral tweets for {} is {} %'.format(stock, (neutral/len(results))*100))
    print('Negative tweets for {} is {} %'.format(stock, (negative/len(results))*100))

Positive tweets for $TERA is 52.38095238095239 %
Neutral tweets for $TERA is 28.57142857142857 %
Negative tweets for $TERA is 0.0 %
Positive tweets for $ARMC is 35.15625 %
Neutral tweets for $ARMC is 42.1875 %
Negative tweets for $ARMC is 6.25 %
Positive tweets for $GTMM is 0.0 %
Neutral tweets for $GTMM is 100.0 %
Negative tweets for $GTMM is 0.0 %
Positive tweets for $GNCP is 23.5 %
Neutral tweets for $GNCP is 37.0 %
Negative tweets for $GNCP is 8.0 %
Positive tweets for $GLAE is 0.0 %
Neutral tweets for $GLAE is 60.0 %
Negative tweets for $GLAE is 0.0 %
Positive tweets for $CRAY is 38.46153846153847 %
Neutral tweets for $CRAY is 45.64102564102564 %
Negative tweets for $CRAY is 4.615384615384616 %
Positive tweets for $GPRO is 38.70967741935484 %
Neutral tweets for $GPRO is 49.193548387096776 %
Negative tweets for $GPRO is 9.67741935483871 %
Positive tweets for $SWCH is 24.324324324324326 %
Neutral tweets for $SWCH is 63.51351351351351 %
Negative tweets for $SWCH is 9.45945945945946 %

# Tweets Per Hour

In [ ]:
twitter_api = oauth_login()
stock_names = ['$TERA', '$ARMC', '$GTMM', '$GNCP', '$GLAE', '$CRAY', '$GPRO', '$SWCH', '$SCKT']
for stock in stock_names:
    q = stock
    results = twitter_search(twitter_api, q, max_results=100)